In [13]:
import psycopg2
import pandas as pd

# Database connection details
username = 'test'
password = 'test123'
hostname = '192.168.1.141'
port = '5432'
database_name = 'mydb'

# Establish connection
conn = psycopg2.connect(
    dbname=database_name,
    user=username,
    password=password,
    host=hostname,
    port=port
)

# SQL query

sql_query = "SELECT TO_TIMESTAMP(lasttradetime) AT TIME ZONE 'Asia/Kolkata' AS time,open, high, low, close,tradedqty as volume FROM whole_indices_futures WHERE instrumentidentifier = 'BANKNIFTY24APR24FUT'AND TO_TIMESTAMP(lasttradetime) AT TIME ZONE 'Asia/Kolkata' <= '2024-04-09 15:30:00' ORDER BY time;"
# Execute SQL query
cursor = conn.cursor()
cursor.execute(sql_query)


# Fetch all the results
rows = cursor.fetchall()


# Convert to DataFrame
df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])

# Close cursor and connection
cursor.close()
conn.close()

# Display the DataFrame
print(df)


                     time      open      high       low     close  volume
0     2024-03-01 09:15:00  46868.90  47152.10  46868.90  47131.80    3885
1     2024-03-01 09:16:00  47131.80  47160.30  47112.60  47114.10    3600
2     2024-03-01 09:17:00  47114.10  47131.00  47114.10  47120.00    1035
3     2024-03-01 09:18:00  47118.45  47150.00  47117.15  47117.65    1455
4     2024-03-01 09:19:00  47117.65  47133.90  47113.65  47125.20    1095
...                   ...       ...       ...       ...       ...     ...
10738 2024-04-09 15:26:00  48812.55  48816.35  48805.00  48810.00    4920
10739 2024-04-09 15:27:00  48810.00  48820.00  48808.50  48812.70    9090
10740 2024-04-09 15:28:00  48812.35  48818.05  48805.00  48805.00    7530
10741 2024-04-09 15:29:00  48805.00  48817.95  48805.00  48805.05    6270
10742 2024-04-09 15:30:00  48809.75  48809.75  48809.75  48809.75      90

[10743 rows x 6 columns]


In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense
df=pd.read_csv('nifty_dataCEE.csv')

In [2]:
df.set_index('lasttradetime',inplace=True)

In [3]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
lookback_window =3
features = 6
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window - 1): 
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, 0:6]) 
    return np.array(X), np.array(y)
model = Sequential()
model.add(LSTM(units=200, return_sequences=True, input_shape=(lookback_window, features)))
model.add(LSTM(units=200))
model.add(Dense(200, activation=tf.nn.leaky_relu))
model.add(Dense(200, activation=tf.nn.leaky_relu))
model.add(Dense(units=6)) 
model.compile(loss='mean_squared_error', optimizer='adam')
X, y = create_sequences(df_scaled, lookback_window)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]




model.fit(X_train, y_train, epochs=50, batch_size=375, validation_data=(X_test, y_test))
y_pred = model.predict(X_test)
y_pred_original = scaler.inverse_transform(y_pred)
data = pd.DataFrame(y_pred_original, columns=['open', 'high', 'low' ,'close','openinterest','tradedqty'])
data 


Epoch 1/50


c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 129ms/step - loss: 0.1847 - val_loss: 0.1038
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0445 - val_loss: 0.1404
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0222 - val_loss: 0.1065
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0142 - val_loss: 0.1088
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0053 - val_loss: 0.1240
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0062 - val_loss: 0.1133
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0022 - val_loss: 0.1074
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0032 - val_loss: 0.1104
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0020 - val_loss: 0.1079
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0017 - val_loss: 0.1030
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0018 - val_loss: 0.1014
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0015 - val_loss: 0.0991
Epoch 13/50

,open,high,low,close,openinterest,tradedqty
0,235.498550,236.291321,234.490341,235.754166,14031.974609,245.052673
1,237.143723,237.908783,236.123276,237.355103,13670.179688,241.048035
2,239.678635,240.393158,238.641479,239.836700,13121.293945,234.290939
3,241.297989,241.929688,240.238892,241.454346,12795.217773,229.414520
4,241.731628,242.348770,240.683228,241.893692,12681.416016,227.263306
...,...,...,...,...,...,...
423,130.375839,128.075745,128.376465,128.776184,36809.332031,768.913208
424,128.964645,126.798531,127.468483,127.582825,37252.218750,779.960938
425,127.704704,125.489067,126.376633,126.466331,37638.812500,790.657959
426,125.672516,123.467888,124.861549,124.809517,38209.800781,789.981140


In [12]:
data

,open,high,low,close,openinterest,tradedqty
0,122.899559,124.462349,122.416138,123.860626,47556.226562,619.499390
1,121.380913,123.001968,120.995125,122.531319,47302.996094,610.034241
2,119.953522,121.601036,119.644196,121.284912,47027.988281,602.043213
3,119.889435,121.500130,119.571915,121.195618,46660.792969,582.024658
4,120.188911,121.807045,119.869286,121.477074,46870.019531,580.046753
...,...,...,...,...,...,...
423,205.515945,207.973984,200.577637,203.169403,150224.171875,2321.743408
424,205.763657,208.238815,200.792007,203.411194,150896.640625,2338.040039
425,206.324020,208.843658,201.272491,203.916336,152339.734375,2385.723633
426,207.234940,209.805695,202.123306,204.796280,154017.921875,2423.076904


In [4]:
import numpy as np
import pandas as pd

# Define the function to generate predictions
def generate_predictions(model, last_sequence, num_predictions):
    predictions = []
    current_sequence = last_sequence.reshape(1, lookback_window, features)  # Reshape to match model input shape
    for _ in range(num_predictions):
        next_prediction = model.predict(current_sequence)  # Predict the next data point
        predictions.append(next_prediction[0])  # Append the prediction to the list of predictions
        current_sequence = np.append(current_sequence[:, 1:, :], next_prediction.reshape(1, 1, features), axis=1)  # Update the current sequence
    return predictions

# Generate predictions for the next 720 data points
last_sequence = X_test[-1]  # Using the last sequence from the test data
num_predictions = 375
predicted_data = generate_predictions(model, last_sequence, num_predictions)

# Inverse transform the predictions to obtain the original scale
predicted_data_original = scaler.inverse_transform(predicted_data)

# Create a DataFrame with the predicted values and column names
predicted_df = pd.DataFrame(predicted_data_original, columns=['open', 'high', 'low', 'close','openinterest','volume'])

# Assuming the last timestamp in the existing data is '2024-04-09 15:30:00', and you want to generate predictions for the next 720 minutes (12 hours)
last_timestamp = pd.Timestamp('2024-04-09 15:30:00')
predicted_df['time'] = pd.date_range(start=last_timestamp + pd.Timedelta(minutes=1), periods=num_predictions, freq='T')

# Print the predicted data
print(predicted_df)
print(predicted_df['close'].max(),predicted_df['close'].min())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━

C:\Users\santh\AppData\Local\Temp\ipykernel_13516\2149060328.py:27: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  predicted_df['time'] = pd.date_range(start=last_timestamp + pd.Timedelta(minutes=1), periods=num_predictions, freq='T')


261.55566887557507 123.86512746252119


In [16]:
predicted_data

[array([0.9059479 , 0.9069565 , 0.907678  , 0.90419954, 0.06263082],
       dtype=float32),
 array([0.9063566 , 0.90747327, 0.908155  , 0.90461165, 0.06250859],
       dtype=float32),
 array([0.9070973 , 0.9080872 , 0.908753  , 0.9052822 , 0.06062227],
       dtype=float32),
 array([0.90879685, 0.9097292 , 0.9103793 , 0.9069223 , 0.05910339],
       dtype=float32),
 array([0.9096278 , 0.91051096, 0.91115934, 0.9077138 , 0.05787962],
       dtype=float32),
 array([0.910714  , 0.9115473 , 0.9121878 , 0.90876067, 0.05672432],
       dtype=float32),
 array([0.91196924, 0.91276205, 0.9133918 , 0.9099815 , 0.05568562],
       dtype=float32),
 array([0.9129904 , 0.91374546, 0.9143654 , 0.91097057, 0.05475242],
       dtype=float32),
 array([0.91412777, 0.9148496 , 0.91545624, 0.912078  , 0.05391004],
       dtype=float32),
 array([0.9152903 , 0.91596913, 0.91655695, 0.9132051 , 0.05315305],
       dtype=float32),
 array([0.9163983 , 0.9170393 , 0.91760796, 0.9142808 , 0.05246928],
       dtyp

In [31]:
    import psycopg2
    import pandas as pd
    import numpy as np
    from datetime import datetime,timedelta
    from keras.models import Sequential
    from keras.layers import LSTM, Dense
    from sklearn.preprocessing import MinMaxScaler
    import logging
    import time

    start_time =time.time()
    # Connect to your PostgreSQL database
    conn = psycopg2.connect(
        dbname="mydb",
        user="test",
        password="test123",
        host="192.168.1.141",
        port="5432"
    )

    # Create a cursor object using the cursor() method
    cursor = conn.cursor()

    instrument_query=f"select distinct instrumentidentifier from whole_indices_futures where instrumentidentifier ='NIFTY18APR2422750PE';"
    cursor.execute(instrument_query)
    instrument_identifiers = [row[0] for row in cursor.fetchall()]
    print("instrument_identifiers",instrument_identifiers)
    print("Number of instrument identifiers:", len(instrument_identifiers))

    cursor.close()

    success =True
    failed_identifiers =[]
    for instrument_identifier in instrument_identifiers:
        try:
            cursor =conn.cursor()

            # Calculate tomorrow's date and time
            tomorrow = datetime.now()
            insert_time = tomorrow.strftime('%Y-%m-%d 09:15:00+00')
            insert_time_dt = datetime.strptime(insert_time, '%Y-%m-%d %H:%M:%S+00')
            num_minutes = 365  # For example, 5 minutes of data
            timestamps = [insert_time_dt + timedelta(minutes=i) for i in range(num_minutes)]
            
            sql_query = f"SELECT TO_TIMESTAMP(lasttradetime) AT TIME ZONE 'Asia/Kolkata' AS time, open, high, low, close, tradedqty as volume FROM whole_indices_futures WHERE instrumentidentifier = '{instrument_identifier}' and TO_TIMESTAMP(lasttradetime) AT TIME ZONE 'Asia/Kolkata' <= '2024-04-09 15:30:00' ORDER BY time;"
            cursor.execute(sql_query)
            rows=cursor.fetchall()
        #========================================================================
            df=pd.DataFrame(rows,columns=[desc[0] for desc in cursor.description])
            df.set_index('time', inplace=True)
            df.fillna(method="ffill", inplace=True)	
            scaler = MinMaxScaler()
            df_scaled = scaler.fit_transform(df)
            lookback_window =3
            features = 5
            def create_sequences(data, lookback_window):
                X, y = [], []
                for i in range(len(data) - lookback_window - 1): 
                    X.append(data[i:(i + lookback_window), :])
                    y.append(data[i + lookback_window, 0:5]) 
                return np.array(X), np.array(y)
            model = Sequential()
            model.add(LSTM(units=200, return_sequences=True, input_shape=(lookback_window, features)))
            model.add(LSTM(units=200))
            model.add(Dense(200, activation=tf.nn.leaky_relu))
            model.add(Dense(200, activation=tf.nn.leaky_relu))
            model.add(Dense(units=5)) 
            model.compile(loss='mean_squared_error', optimizer='adam')
            X, y = create_sequences(df_scaled, lookback_window)
            train_size = int(len(X) * 0.8)
            X_train, X_test = X[:train_size], X[train_size:]
            y_train, y_test = y[:train_size], y[train_size:]




            model.fit(X_train, y_train, epochs=50, batch_size=375, validation_data=(X_test, y_test))
            y_pred = model.predict(X_test)
            y_pred_original = scaler.inverse_transform(y_pred)
            data = pd.DataFrame(y_pred_original, columns=['open', 'high', 'low' ,'close','volume'])
            data 

            def generate_predictions(model, last_sequence, num_predictions):
                predictions = []
                current_sequence = last_sequence.reshape(1, lookback_window, features)  # Reshape to match model input shape
                for _ in range(num_predictions):
                    next_prediction = model.predict(current_sequence)  # Predict the next data point
                    predictions.append(next_prediction[0])  # Append the prediction to the list of predictions
                    current_sequence = np.append(current_sequence[:, 1:, :], next_prediction.reshape(1, 1, features), axis=1)  # Update the current sequence
                return predictions


            last_sequence = X_test[-1]  # Using the last sequence from the test data
            num_predictions = 720
            predicted_data = generate_predictions(model, last_sequence, num_predictions)

            # Inverse transform the predictions to obtain the original scale
            predicted_data_original = scaler.inverse_transform(predicted_data)

            # Create a DataFrame with the predicted values and column names
            predicted_df = pd.DataFrame(predicted_data_original, columns=['open', 'high', 'low', 'close', 'volume'])

            # Assuming the last timestamp in the existing data is '2024-04-09 15:30:00', and you want to generate predictions for the next 720 minutes (12 hours)
            last_timestamp = pd.Timestamp('2024-04-09 15:30:00')
            predicted_df['time'] = pd.date_range(start=last_timestamp + pd.Timedelta(minutes=1), periods=num_predictions, freq='T')

            df=predicted_df

        #========================================================
            # Insert data into the table
            for timestamp, row in zip(timestamps, predicted_df.itertuples(index=False)):
                insert_query =f"INSERT INTO santhosh1 (lasttradetime,open, high, low, close,instrumentidentifier,volume) VALUES (%s, %s, %s, %s, %s, %s, %s)"
                cursor.execute(insert_query, (timestamp,row.open, row.high, row.low, row.close , instrument_identifier,row.volume))
            print("inserted successfully")

            conn.commit()
            cursor.close()

        except Exception as e:
            success = False
            failed_identifiers.append(instrument_identifier)
            print(f"Error occurred for instrument identifier: {instrument_identifier}, Error: {str(e)}")
            continue

    conn.close()

    if success:
        print("All instrument identifiers processed successfully.")
    else:
        print("Some instrument identifiers failed to process.")
        print("Failed instrument identifiers:", failed_identifiers)
        # Log failed instrument identifiers to a file
        log_error_instrument = datetime.now().strftime("%Y-%m-%d") + "_failed_instrument_identifiers.log"
        logging.basicConfig(filename=log_error_instrument, level=logging.INFO)
        logging.info("Failed instrument identifiers: " + str(failed_identifiers))

    end_time = time.time()
    print("end_time",end_time)
    print("start_time",start_time)
    execution_time =end_time -start_time
    print("execution_time",execution_time)

instrument_identifiers ['NIFTY18APR2422800PE']
Number of instrument identifiers: 1


C:\Users\santh\AppData\Local\Temp\ipykernel_18880\3385708768.py:51: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 192ms/step - loss: 0.2479 - val_loss: 0.0064
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0664 - val_loss: 0.0465
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0307 - val_loss: 0.0095
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0209 - val_loss: 0.0127
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0071 - val_loss: 0.0314
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0093 - val_loss: 0.0204
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0041 - val_loss: 0.0140
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0046 - val_loss: 0.0188
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.0035 - val_loss: 0.0169
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0024 - val_loss: 0.0120
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0025 - val_loss: 0.0119
Epoch 12/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0018 - val_loss: 0.0101


C:\Users\santh\AppData\Local\Temp\ipykernel_18880\3385708768.py:105: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  predicted_df['time'] = pd.date_range(start=last_timestamp + pd.Timedelta(minutes=1), periods=num_predictions, freq='T')


inserted successfully
All instrument identifiers processed successfully.
end_time 1712925128.2720711
start_time 1712925042.0679834
execution_time 86.20408773422241


In [34]:
import psycopg2
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import logging
import time

start_time =time.time()
# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="mydb",
    user="test",
    password="test123",
    host="192.168.1.141",
    port="5432"
)

# Create a cursor object using the cursor() method
cursor = conn.cursor()

instrument_query=f"select distinct instrumentidentifier from whole_indices_futures where instrumentidentifier like '%NIFTY18APR242%' limit 1"
cursor.execute(instrument_query)
instrument_identifiers = [row[0] for row in cursor.fetchall()]
print("instrument_identifiers",instrument_identifiers)
print("Number of instrument identifiers:", len(instrument_identifiers))

cursor.close()

success =True
failed_identifiers =[]
for instrument_identifier in instrument_identifiers:
    try:
        cursor =conn.cursor()

        # Calculate tomorrow's date and time
        tomorrow = datetime.now()
        insert_time = tomorrow.strftime('%Y-%m-%d 09:15:00+00')
        insert_time_dt = datetime.strptime(insert_time, '%Y-%m-%d %H:%M:%S+00')
        num_minutes = 365  # For example, 5 minutes of data
        timestamps = [insert_time_dt + timedelta(minutes=i) for i in range(num_minutes)]

        sql_query = f"SELECT TO_TIMESTAMP(lasttradetime) AT TIME ZONE 'Asia/Kolkata' AS time, open, high, low, close FROM whole_indices_futures WHERE instrumentidentifier = '{instrument_identifier}' and TO_TIMESTAMP(lasttradetime) BETWEEN CURRENT_DATE - interval '2 day' + TIME '09:15:00' AND CURRENT_DATE + TIME '15:30:00'"
        cursor.execute(sql_query)
        rows=cursor.fetchall()
    #========================================================================
        df=pd.DataFrame(rows,columns=[desc[0] for desc in cursor.description])
        df.set_index('time', inplace=True)
        df.fillna(method="ffill", inplace=True)	
        scaler = MinMaxScaler(feature_range=(0, 1))
        df_scaled = scaler.fit_transform(df)
        lookback_window = 3
        features = 4

        def create_sequences(data, lookback_window):
            X, y = [], []
            for i in range(len(data) - lookback_window - 1): 
                X.append(data[i:(i + lookback_window), :])
                y.append(data[i + lookback_window, 0:4]) 
            return np.array(X), np.array(y)

        model = Sequential()
        model.add(LSTM(units=50, return_sequences=True, input_shape=(lookback_window, features)))
        model.add(LSTM(units=50))
        model.add(Dense(units=4)) 
        model.compile(loss='mean_squared_error', optimizer='adam')

        X, y = create_sequences(df_scaled, lookback_window)
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
        y_pred = model.predict(X_test)
        y_pred_original = scaler.inverse_transform(y_pred)
        data = pd.DataFrame(y_pred_original, columns=['open', 'high', 'low' ,'close'])

        def generate_predictions(model, last_sequence, num_predictions):
            predictions = []
            current_sequence = last_sequence.reshape(1, lookback_window, features)  # Reshape to match model input shape
            for _ in range(num_predictions):
                next_prediction = model.predict(current_sequence)  
                predictions.append(next_prediction[0]) 
                current_sequence = np.append(current_sequence[:, 1:, :], next_prediction.reshape(1, 1, features), axis=1)  # Update the current sequence
            return predictions

        last_sequence = X_test[-1]  # Using the last sequence from the test data
        num_predictions = 365
        predicted_data = generate_predictions(model, last_sequence, num_predictions)
        predicted_data_original = scaler.inverse_transform(predicted_data)
        predicted_df = pd.DataFrame(predicted_data_original, columns=['open', 'high', 'low', 'close'])
        print(predicted_df)
        df=predicted_df

    #========================================================
        # Insert data into the table
        for timestamp, row in zip(timestamps, predicted_df.itertuples(index=False)):
            insert_query =f"INSERT INTO onetwothree (lasttradetime,open, high, low, close,instrumentidentifier) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(insert_query, (timestamp,row.open, row.high, row.low, row.close , instrument_identifier))
        print("inserted successfully")

        conn.commit()
        cursor.close()

    except Exception as e:
        success = False
        failed_identifiers.append(instrument_identifier)
        print(f"Error occurred for instrument identifier: {instrument_identifier}, Error: {str(e)}")
        continue

conn.close()

if success:
    print("All instrument identifiers processed successfully.")
else:
    print("Some instrument identifiers failed to process.")
    print("Failed instrument identifiers:", failed_identifiers)
    # Log failed instrument identifiers to a file
    log_error_instrument = datetime.now().strftime("%Y-%m-%d") + "_failed_instrument_identifiers.log"
    logging.basicConfig(filename=log_error_instrument, level=logging.INFO)
    logging.info("Failed instrument identifiers: " + str(failed_identifiers))

end_time = time.time()
execution_time =start_time -end_time
print("execution_time",execution_time)


instrument_identifiers ['NIFTY18APR2420050CE']
Number of instrument identifiers: 1
Epoch 1/50


C:\Users\santh\AppData\Local\Temp\ipykernel_18880\383107645.py:51: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)
c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 12/5